In [1]:
import os

import tensorflow as tf

In [2]:
print(tf.__version__)

2.1.0


In [3]:
tf.random.set_seed(0)

In [4]:
path_to_zip = tf.keras.utils.get_file(
    fname='cats_and_dogs_filtered.zip',
    origin='https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip',
    extract=True
)

In [5]:
path_to_dir = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [6]:
train_dir = os.path.join(path_to_dir, 'train')
test_dir = os.path.join(path_to_dir, 'validation')

In [7]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

In [8]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [9]:
batch_size = 64
height = 160
width = 160

In [10]:
train_generator = train_datagen.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    target_size=(height, width),
    class_mode='binary',
    subset='training'
)

Found 1600 images belonging to 2 classes.


In [11]:
valid_generator = train_datagen.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    target_size=(height, width),
    class_mode='binary',
    subset='validation'
)

Found 400 images belonging to 2 classes.


In [12]:
test_generator = test_datagen.flow_from_directory(
    batch_size=batch_size,
    directory=test_dir,
    target_size=(height, width),
    class_mode='binary'
)

Found 1000 images belonging to 2 classes.


In [13]:
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    weights='imagenet', input_shape=(height, width, 3),
    include_top=False, pooling='avg'
)

In [14]:
x = base_model.output
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

In [15]:
model = tf.keras.Model(inputs=base_model.input, outputs=x)

In [16]:
print(len(model.layers))

157


In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [18]:
print(model.layers[0] is base_model.layers[0])

True


In [19]:
print(base_model.layers[0].trainable)

True


In [20]:
print(model.layers[0].trainable)

True


In [21]:
base_model.trainable = False

In [22]:
print(base_model.layers[0].trainable)

False


In [23]:
print(model.layers[0].trainable)

False


In [24]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
print(model.evaluate(test_generator, verbose=0))

  ...
    to  
  ['...']
[0.7827205918729305, 0.473]


In [26]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // batch_size,
    epochs=6
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 25 steps, validate for 6 steps
Epoch 1/6
25/25 [==============================] - 4s 164ms/step - loss: 0.7280 - accuracy: 0.5475 - val_loss: 0.6718 - val_accuracy: 0.6120
Epoch 2/6
25/25 [==============================] - 2s 85ms/step - loss: 0.6413 - accuracy: 0.6306 - val_loss: 0.5767 - val_accuracy: 0.7292
Epoch 3/6
25/25 [==============================] - 2s 84ms/step - loss: 0.5740 - accuracy: 0.7044 - val_loss: 0.4978 - val_accuracy: 0.7917
Epoch 4/6
25/25 [==============================] - 2s 85ms/step - loss: 0.5162 - accuracy: 0.7600 - val_loss: 0.4349 - val_accuracy: 0.8385
Epoch 5/6
25/25 [==============================] - 2s 87ms/step - loss: 0.4672 - accuracy: 0.8012 - val_loss: 0.3834 - val_accuracy: 0.8620
Epoch 6/6
25/25 [==============================] - 2s 86ms/step - loss: 0.4255 - accuracy: 0.8350 - val_loss: 0.3423 - val_accuracy: 0.8750


In [27]:
print(model.evaluate(test_generator, verbose=0))

  ...
    to  
  ['...']
[0.3087761905044317, 0.912]


In [28]:
idx = [l.name for l in base_model.layers].index('block_12_expand')

In [29]:
for layer in base_model.layers[idx:]:
    layer.trainable = True

In [30]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // batch_size,
    epochs=6
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 25 steps, validate for 6 steps
Epoch 1/6
25/25 [==============================] - 5s 208ms/step - loss: 0.3144 - accuracy: 0.8944 - val_loss: 0.2101 - val_accuracy: 0.9375
Epoch 2/6
25/25 [==============================] - 2s 86ms/step - loss: 0.2024 - accuracy: 0.9538 - val_loss: 0.1632 - val_accuracy: 0.9453
Epoch 3/6
25/25 [==============================] - 2s 87ms/step - loss: 0.1433 - accuracy: 0.9762 - val_loss: 0.1352 - val_accuracy: 0.9505
Epoch 4/6
25/25 [==============================] - 2s 87ms/step - loss: 0.1029 - accuracy: 0.9869 - val_loss: 0.1181 - val_accuracy: 0.9609
Epoch 5/6
25/25 [==============================] - 2s 92ms/step - loss: 0.0744 - accuracy: 0.9944 - val_loss: 0.1072 - val_accuracy: 0.9661
Epoch 6/6
25/25 [==============================] - 2s 87ms/step - loss: 0.0540 - accuracy: 0.9975 - val_loss: 0.1002 - val_accuracy: 0.9688


In [32]:
print(model.evaluate(test_generator, verbose=0))

  ...
    to  
  ['...']
[0.06722519337199628, 0.978]
